<a href="https://colab.research.google.com/github/skyprince999/AV-GOT-Ship-Identification/blob/master/AV_GoT_Ship_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import sys

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
print("TensorFlow version is ", tf.__version__)

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import PIL.Image as Image
from IPython.display import clear_output

TensorFlow version is  1.13.1


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
sys.path.insert(0, '/content/gdrive/My Drive/Colab Notebooks/my_modules')
import unzipFiles

In [0]:
PATH = '/content/gdrive/My Drive/AV_GoT_Ships/'

categories = {'Cargo': 1, 
'Military': 2, 
'Carrier': 3, 
'Cruise': 4, 
'Tankers': 5}

os.listdir(PATH)

['train.zip',
 'test.csv',
 'sample_submission.csv',
 'models',
 'submission_mobilenet_25May2019_2335.csv',
 'training_1',
 'training_2',
 'training_3',
 'models_v2']

In [0]:
unzipFiles.copyTest(PATH)
unzipFiles.unzipData(PATH)
unzipFiles.copyTrainValid(categories)

['.config', 'sample_submission.csv', 'gdrive', 'test.csv', 'sample_data']
Unzipped Successful
['.config', '__MACOSX', 'sample_submission.csv', 'gdrive', 'images', 'train.csv', 'test.csv', 'sample_data']
['.config', '__MACOSX', 'sample_submission.csv', 'gdrive', 'images', 'train.csv', 'test.csv', 'sample_data']
Shape of train file:  (6252, 2)
Number of categories:  5


0

In [0]:
os.listdir('train')

['2', '4', '3', '1', '5']

In [0]:
# Download random 10 images from each class

trainDF = pd.read_csv('train.csv')
print(trainDF.shape)

(6252, 2)


In [0]:
from google.colab import files

In [0]:
for i in range(1,6):
  categoryDF = trainDF[ trainDF['category'] == i]
  sample = categoryDF.sample( n=15, random_state = 987)
  print(i)
  print(sample.shape)
  for filename in sample['image']:
    filePath = 'train/' + str(i) +'/' + filename
    try:
      files.download(filePath)
    except:
      print('Error: Unable to find file: ', filename)
    
  input("Press enter to continue....")
    

1
(15, 2)
Error: Unable to find file:  2870005.jpg
Error: Unable to find file:  2835454.jpg
Error: Unable to find file:  2792316.jpg
Error: Unable to find file:  2892555.jpg
Press enter to continue....
2
(15, 2)
Error: Unable to find file:  2743443.jpg
Error: Unable to find file:  2885510.jpg
Error: Unable to find file:  964374.jpg
Error: Unable to find file:  2456597.jpg
Press enter to continue....
3
(15, 2)
Error: Unable to find file:  2901437.jpg
Press enter to continue....
4
(15, 2)
Error: Unable to find file:  627875.jpg
Error: Unable to find file:  1073645.jpg
Error: Unable to find file:  2792128.jpg
Error: Unable to find file:  906470.jpg
Error: Unable to find file:  1788614.jpg
Error: Unable to find file:  2880420.jpg
Error: Unable to find file:  2494294.jpg
Error: Unable to find file:  1820874.jpg
Press enter to continue....
5
(15, 2)
Error: Unable to find file:  2796339.jpg
Error: Unable to find file:  2818899.jpg
Error: Unable to find file:  2892419.jpg
Press enter to contin

In [0]:
!pip install azure-cognitiveservices-vision-customvision

     |████████████████████████████████| 112kB 2.8MB/s 
     |████████████████████████████████| 81kB 29.9MB/s 
     |████████████████████████████████| 51kB 27.2MB/s 


In [0]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateEntry

ENDPOINT = "https://southcentralus.api.cognitive.microsoft.com/customvision/v3.0/Prediction/8729b181-d35b-4ea8-b104-205955596e9b/classify/iterations/Iteration1/image"

# Replace with a valid key
training_key = "983a4aa1d0104e4ba8fe6c8de32d341d"
prediction_key = "587bf9e9007e43e3a9033c04f97cf78f"
prediction_resource_id = "/subscriptions/15dc4e98-a0cf-4466-a045-0dcfcaaf85b1/resourceGroups/ship_identification/providers/Microsoft.CognitiveServices/accounts/ship_identification_prediction"

publish_iteration_name = "Iteration1"
projectID = '8729b181-d35b-4ea8-b104-205955596e9b'
trainer = CustomVisionTrainingClient(training_key, endpoint=ENDPOINT)

In [0]:
trainer.get_project(projectID)

CustomVisionErrorException: ignored

In [0]:
testDF = pd.read_csv('test.csv')
print(testDF.shape)

(2680, 1)


In [0]:
len(os.listdir('images'))

8932

In [0]:
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient

# Now there is a trained endpoint that can be used to make a prediction
predictor = CustomVisionPredictionClient(prediction_key, endpoint=ENDPOINT)
predictions = list()

In [0]:
for idx, row in testDF.iterrows():
  filename = row['image']
  with open('images/' + filename, "rb") as image_contents:
      results = predictor.classify_image(project.id, publish_iteration_name, image_contents.read())

      # Display the results.
      for prediction in results.predictions:
          print ("\t" + prediction.tag_name + ": {0:.2f}%".format(prediction.probability * 100))
      predictions.append(prediction)
      break

NameError: ignored

In [0]:
trainDF[trainDF['image'] == '2870005.jpg']

,image,category
3429,2870005.jpg,1


In [0]:
image_size = 160 # All images will be resized to 160x160
batch_size = 32
train_dir = 'train'
validation_dir = 'valid'


# Rescale all images by 1./255 and apply image augmentation
train_datagen = keras.preprocessing.image.ImageDataGenerator(
                rescale=1./255)

validation_datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [0]:
# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
                train_dir,  # Source directory for the training images
                target_size=(image_size, image_size),
                batch_size=batch_size, seed = 999,
                # Since we use binary_crossentropy loss, we need binary labels
                class_mode='categorical')

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = validation_datagen.flow_from_directory(
                validation_dir, # Source directory for the validation images
                target_size=(image_size, image_size),
                batch_size=batch_size, seed = 999,
                class_mode='categorical')

Found 4999 images belonging to 5 classes.
Found 1253 images belonging to 5 classes.


In [0]:
IMG_SHAPE = (image_size, image_size, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

Instructions for updating:
Colocations handled automatically by placer.
9412608/9406464 [==============================] - 2s 0us/step


In [0]:
base_model.trainable = False

In [0]:
# Let's take a look at the base model architecture
base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 160, 160, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 161, 161, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 80, 80, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalizationV1) (None, 80, 80, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

In [0]:
model = tf.keras.Sequential([
  base_model,
  keras.layers.GlobalAveragePooling2D(),
  keras.layers.Dense(5, activation='relu')
])

In [0]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr = 0.0001),
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_160 (Model) (None, 5, 5, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 5)                 6405      
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________


In [0]:
len(model.trainable_variables)

2

In [0]:
epochs = 50
steps_per_epoch = train_generator.n // batch_size
validation_steps = validation_generator.n // batch_size

In [0]:
checkpoint_path = PATH + "training_1/cp_v3.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create checkpoint callback
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path, period = 2,
                                                 verbose=1)

In [0]:
history = model.fit_generator(train_generator,
                              steps_per_epoch = steps_per_epoch,
                              epochs=epochs,
                              workers=4,
                              validation_data=validation_generator,
                              validation_steps=validation_steps,
                             callbacks = [cp_callback])

Epoch 1/50
157/157 [==============================] - 11s 72ms/step - loss: 0.6905 - categorical_accuracy: 0.7848 - val_loss: 0.9609 - val_categorical_accuracy: 0.7550
Epoch 2/50
40/40 [==============================] - 2s 59ms/step - loss: 1.0195 - categorical_accuracy: 0.7526

Epoch 00002: saving model to /content/gdrive/My Drive/AV_GoT_Ships/training_1/cp_v2.ckpt
157/157 [==============================] - 12s 76ms/step - loss: 0.6828 - categorical_accuracy: 0.7966 - val_loss: 1.0195 - val_categorical_accuracy: 0.7526
Epoch 3/50
157/157 [==============================] - 11s 71ms/step - loss: 0.7132 - categorical_accuracy: 0.7834 - val_loss: 1.0067 - val_categorical_accuracy: 0.7526
Epoch 4/50
40/40 [==============================] - 3s 68ms/step - loss: 0.9670 - categorical_accuracy: 0.7598

Epoch 00004: saving model to /content/gdrive/My Drive/AV_GoT_Ships/training_1/cp_v2.ckpt
157/157 [==============================] - 12s 80ms/step - loss: 0.6925 - categorical_accuracy: 0.7954 - 

In [0]:
model.save(PATH + 'models_v2/epoch50B_26May2019_1600.h5')